### Import libraries

In [1]:
from IPython.core.display import display
from src.utils.preprocessing import preprocessing_pipeline
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

### Load Data

In [3]:
DATA_PATH = '../../../data'
X_fine, y_fine = import_data(DATA_PATH, segmentation_type='fine',
                             drop_user_features=True,
                             drop_expert=True)

In [4]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='fine',
                       drop_user_features=True,
                       drop_expert=False)

In [5]:
display(X_fine.head())
display(y_fine.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               6.0          6.0   
                                     1               3.0          3.0   
                                     2               4.0          4.0   
                                     3               4.0          4.0   
                                     4               4.0          4.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          4.0   
                                     1                3.0          4.0   
                                     2                3.0          4.0   
                                     3                4.0          4.0   
                                     4                5.0          4.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          5.0   
                                     1                4.0          5.0   
                                     2                3.0          3.0   
                                     3                4.0          5.0   
                                     4                4.0          5.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                6.0          6.0   
                                     1                4.0          4.0   
                                     2                4.0          4.0   
                                     3                3.0          4.0   
                                     4                3.0          4.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          6.0  ...   
                                     1                4.0          3.0  ...   
                                     2                4.0          4.0  ...   
                                     3                4.0          3.0  ...   
                                     4                4.0          3.0  ...   

                                              MFCC_std11  MFCC_std12  \
subject                              file_id                           
008ba489-31ad-44d8-856b-fcf72369dc46 0         12.109140   11.702864   
                                     1         10.000873    8.967152   
                                     2          8.318832    8.688411   
                                     3         13.294838    8.694027   
                                     4         12.814457    8.501382   

                                              Crest_Factor  Cough_Length  \
subject                              file_id                               
008ba489-31ad-44d8-856b-fcf72369dc46 0            7.649371      0.767250   
                                     1            8.001334      0.568917   
                                     2           11.189755      0.581667   
                                     3            7.840153      0.540250   
                                     4            7.869070      0.546333   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.398299     0.026407   
                                     1           0.230052     0.086242   
                                     2           0.105747     0.180344   
               

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
                                     2          1.0
                                     3          1.0
                                     4          1.0

### Preprocessing

In [6]:
X_fine = preprocessing_pipeline(X_fine, stop=None, dummy=False)

In [7]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = preprocessing_pipeline(X_e, stop=None, dummy=False)
X_e['Expert'] = expert.values

### Split expert models

In [8]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

In [9]:
DECISION_METRIC = 'roc_auc_score'
CV_K = 10

#### 1. Logistic regression

In [10]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  roc_auc_score_var  \
max_iter oversampling                                               
10000    True          0.419980       0.576921           0.000075   
         False         0.108464       0.519784           0.000034   
100000   True          0.419980       0.576921           0.000075   
         False         0.108464       0.519784           0.000034   

                       accuracy_score  explained_variance_score  
max_iter oversampling                                            
10000    True                0.582596                 -0.955107  
         False               0.732715                 -0.078227  
100000   True                0.582596                 -0.955107  
         False               0.732715                 -0.078227

In [11]:
# Best model parameters

best_log = log_results.iloc[[
    log_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.41998,0.576921,0.000075,0.582596,-0.955107


In [12]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)

In [13]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.377582,0.658343,0.000329,0.702593,-1.080326


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.313398,0.560974,0.000271,0.606168,-1.342065


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,False,0.601255,0.634507,0.000249,0.637517,-0.450838


#### 2. Linear Discriminant Analysis

In [14]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.421561,0.577677,0.000070,0.581122,-0.954093
False,0.112790,0.520406,0.000044,0.732287,-0.085119


In [15]:
# Best model parameters

best_lda = lda_results.iloc[[
    lda_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.421561,0.577677,0.00007,0.581122,-0.954093


In [16]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=CV_K,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=CV_K,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=CV_K,
                                         params=LDA_PARAMS)

In [17]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.385895,0.669317,0.000451,0.6953,-1.085271


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.31677,0.563777,0.000419,0.602887,-1.341583


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.618843,0.635322,0.000243,0.635862,-0.461038


#### 3. K-nearest Neighbors

In [18]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.496124       0.645125           0.000084   
            False         0.468184       0.637895           0.000097   
2           True          0.486269       0.642138           0.000149   
            False         0.280390       0.567249           0.000024   
3           True          0.485430       0.627727           0.000129   
            False         0.401914       0.604765           0.000071   
4           True          0.481586       0.629218           0.000145   
            False         0.269152       0.563476           0.000025   
5           True          0.477373       0.614558           0.000078   
            False         0.352587       0.586441           0.000029   
6           True          0.476050       0.618208           0.000033   
            False         0.241762       0.555284           0.000018   
7           True          0.465608       0.597422           0.000048   
            False         0.303400       0.569584           0.000055   
8           True          0.468860       0.606298           0.000036   
            False         0.212700       0.547167           0.000036   
9           True          0.459527       0.587452           0.000059   
            False         0.268416       0.560008           0.000089   
10          True          0.462711       0.596140           0.000052   
            False         0.186862       0.540571           0.000048   
11          True          0.455207       0.580173           0.000053   
            False         0.234826       0.551052           0.000087   
12          True          0.458489       0.588700           0.000040   
            False         0.170119       0.536516           0.000032   
13          True          0.451888       0.574207           0.000017   
            False         0.216186       0.546955           0.000062   
14          True          0.455230       0.582665           0.000024   
            False         0.155706       0.533480           0.000033   
15          True          0.449660       0.569969           0.000025   
            False         0.195018       0.542196           0.000065   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.621969                 -0.692563  
            False               0.721398                 -0.419763  
2           True                0.667998                 -0.630661  
            False               0.744080                 -0.138710  
3           True                0.556871                 -0.719773  
            False               0.725820                 -0.367638  
4           True                0.596101                 -0.757169  
            False               0.742986                 -0.137699  
5           True                0.519306                 -0.698723  
            False               0.732240                 -0.288991  
6           True                0.553543                 -0.765040  
            False               0.741607                 -0.123052  
7           True                0.488350                 -0.700914  
            False               0.733143                 -0.241266  
8           True                0.522349                 -0.760337  
            False               0.740276                 -0.107961  
9           True                0.468569                 -0.687357  
            False               0.735854                 -0.190832  
10          True                0.498906                 -0.750506  
            False               0.739515                 -0.091169  
11          True                0.454731                 -0.676328  
            False               0.736519                 -0.156369  
12          True                0.482073                 -0.73772

In [19]:
# Best model parameters

best_knn = knn_results.iloc[[
    knn_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.496124,0.645125,0.000084,0.621969,-0.692563


In [20]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=KNN_PARAMS)

display(knn_results_1, knn_results_2, knn_results_3)

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.438047       0.730204           0.000176   
            False         0.487560       0.701153           0.000568   
2           True          0.452216       0.725989           0.000255   
            False         0.320324       0.596073           0.000178   
3           True          0.387682       0.696528           0.000343   
            False         0.409470       0.638931           0.000167   
4           True          0.403329       0.705268           0.000140   
            False         0.245159       0.568829           0.000101   
5           True          0.368369       0.680627           0.000459   
            False         0.312604       0.593418           0.000085   
6           True          0.381832       0.691935           0.000348   
            False         0.177202       0.546854           0.000197   
7           True          0.357105       0.669709           0.000318   
            False         0.245005       0.568485           0.000115   
8           True          0.367234       0.679390           0.000213   
            False         0.112524       0.527888           0.000159   
9           True          0.345040       0.654459           0.000346   
            False         0.166634       0.543325           0.000108   
10          True          0.356109       0.667956           0.000383   
            False         0.097332       0.523995           0.000207   
11          True          0.338989       0.646594           0.000346   
            False         0.142131       0.536483           0.000293   
12          True          0.349433       0.660284           0.000417   
            False         0.065192       0.515521           0.000158   
13          True          0.335368       0.641692           0.000290   
            False         0.100715       0.525021           0.000155   
14          True          0.344500       0.654110           0.000423   
            False         0.043237       0.510093           0.000075   
15          True          0.330914       0.635143           0.000277   
            False         0.058639       0.513776           0.000122   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.699028                 -0.923426  
            False               0.841491                 -0.234356  
2           True                0.742301                 -0.788310  
            False               0.866288                  0.050929  
3           True                0.600648                 -1.164439  
            False               0.858995                 -0.062938  
4           True                0.645543                 -1.103386  
            False               0.860778                  0.027352  
5           True                0.543922                 -1.190697  
            False               0.858023                 -0.031317  
6           True                0.584441                 -1.180610  
            False               0.856078                  0.007375  
7           True                0.505186                 -1.163615  
            False               0.857699                 -0.003867  
8           True                0.542464                 -1.192821  
            False               0.851864                 -0.007417  
9           True                0.472285                 -1.147792  
            False               0.853971                 -0.007628  
10          True                0.506483                 -1.175763  
            False               0.851864                 -0.000724  
11          True                0.452512                 -1.121361  
            False               0.853323                 -0.006287  
12          True                0.483955                 -1.15388

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.421320       0.663248           0.000327   
            False         0.384244       0.622260           0.000161   
2           True          0.413072       0.649490           0.000299   
            False         0.165166       0.541012           0.000084   
3           True          0.389677       0.638123           0.000156   
            False         0.262850       0.568104           0.000063   
4           True          0.394244       0.640119           0.000308   
            False         0.101991       0.522356           0.000065   
5           True          0.377518       0.625010           0.000193   
            False         0.157261       0.534215           0.000123   
6           True          0.381337       0.628727           0.000190   
            False         0.053183       0.510840           0.000060   
7           True          0.367027       0.611484           0.000219   
            False         0.089461       0.517797           0.000064   
8           True          0.369155       0.614380           0.000140   
            False         0.031355       0.506183           0.000037   
9           True          0.359373       0.600396           0.000228   
            False         0.048900       0.509021           0.000053   
10          True          0.362144       0.604781           0.000310   
            False         0.015194       0.503379           0.000009   
11          True          0.357519       0.596998           0.000280   
            False         0.028459       0.505287           0.000027   
12          True          0.359788       0.601041           0.000318   
            False         0.009929       0.502194           0.000008   
13          True          0.351127       0.587025           0.000456   
            False         0.012468       0.502302           0.000011   
14          True          0.355156       0.593955           0.000414   
            False         0.002797       0.500626           0.000002   
15          True          0.346665       0.579823           0.000371   
            False         0.008175       0.501520           0.000007   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.673097                 -0.917793  
            False               0.800394                 -0.313874  
2           True                0.716404                 -0.799153  
            False               0.822047                 -0.024134  
3           True                0.574147                 -1.111256  
            False               0.810367                 -0.180494  
4           True                0.620997                 -1.081318  
            False               0.816273                 -0.043785  
5           True                0.520997                 -1.116360  
            False               0.811024                 -0.120202  
6           True                0.558924                 -1.145943  
            False               0.815486                 -0.026847  
7           True                0.480709                 -1.095616  
            False               0.812992                 -0.066279  
8           True                0.513517                 -1.156934  
            False               0.815354                 -0.017194  
9           True                0.453412                 -1.074611  
            False               0.813780                 -0.040368  
10          True                0.485302                 -1.146606  
            False               0.816273                 -0.001269  
11          True                0.435958                 -1.027744  
            False               0.814961                 -0.019753  
12          True                0.462598                 -1.10147

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.689057       0.702560           0.000240   
            False         0.673007       0.696979           0.000099   
2           True          0.574627       0.657116           0.000160   
            False         0.525588       0.644066           0.000128   
3           True          0.674622       0.682086           0.000219   
            False         0.655928       0.686006           0.000179   
4           True          0.623435       0.672524           0.000201   
            False         0.568417       0.658367           0.000128   
5           True          0.668907       0.673858           0.000301   
            False         0.643890       0.676140           0.000262   
6           True          0.626111       0.663639           0.000243   
            False         0.568696       0.650921           0.000175   
7           True          0.661291       0.664807           0.000183   
            False         0.632827       0.669099           0.000198   
8           True          0.630133       0.660134           0.000127   
            False         0.578906       0.655087           0.000148   
9           True          0.656634       0.658508           0.000264   
            False         0.623960       0.663651           0.000245   
10          True          0.633460       0.659120           0.000180   
            False         0.577334       0.651341           0.000244   
11          True          0.647332       0.649044           0.000318   
            False         0.614058       0.656868           0.000220   
12          True          0.630165       0.652544           0.000230   
            False         0.573729       0.647653           0.000182   
13          True          0.647572       0.648131           0.000133   
            False         0.610351       0.653919           0.000166   
14          True          0.633139       0.651843           0.000115   
            False         0.577585       0.647980           0.000110   
15          True          0.648631       0.649247           0.000185   
            False         0.607282       0.652253           0.000134   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.702483                 -0.192144  
            False               0.699034                 -0.205552  
2           True                0.668000                 -0.232080  
            False               0.658483                 -0.177041  
3           True                0.680690                 -0.274806  
            False               0.689103                 -0.242330  
4           True                0.679034                 -0.258461  
            False               0.669931                 -0.201924  
5           True                0.672138                 -0.304909  
            False               0.679586                 -0.279850  
6           True                0.668138                 -0.321358  
            False               0.661379                 -0.263552  
7           True                0.663034                 -0.339242  
            False               0.673241                 -0.301220  
8           True                0.663448                 -0.347206  
            False               0.664690                 -0.262327  
9           True                0.656414                 -0.362748  
            False               0.668138                 -0.317716  
10          True                0.661793                 -0.355664  
            False               0.660690                 -0.286680  
11          True                0.647034                 -0.399389  
            False               0.661793                 -0.341254  
12          True                0.654621                 -0.38481

In [21]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.438047,0.730204,0.000176,0.699028,-0.923426


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.42132,0.663248,0.000327,0.673097,-0.917793


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.689057,0.70256,0.00024,0.702483,-0.192144


#### 4. Support Vector Classifier

In [22]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  roc_auc_score_var  \
kernel gamma oversampling                                               
linear 0.10  True          0.422718       0.574958           0.000139   
             False         0.000000       0.500000           0.000000   
rbf    0.10  True          0.081058       0.517629           0.000012   
             False         0.027558       0.506930           0.000003   
linear 0.01  True          0.422718       0.574958           0.000139   
             False         0.000000       0.500000           0.000000   
rbf    0.01  True          0.426553       0.597194           0.000071   
             False         0.052702       0.511061           0.000005   

                           accuracy_score  explained_variance_score  
kernel gamma oversampling                                            
linear 0.10  True                0.567095                 -0.972731  
             False               0.732049                  0.000000  
rbf    0.10  True                0.737851                 -0.016593  
             False               0.735711                  0.008185  
linear 0.01  True                0.567095                 -0.972731  
             False               0.732049                  0.000000  
rbf    0.01  True                0.645459                 -0.771528  
             False               0.735473                 -0.012809

In [23]:
# Best model parameters

best_svc = svc_results.iloc[[
    svc_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,True,0.426553,0.597194,0.000071,0.645459,-0.771528


In [24]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=SVC_PARAMS)

In [25]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()[DECISION_METRIC].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,True,0.423746,0.67148,0.000368,0.799676,-0.54738


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
linear,0.1,True,0.32164,0.565561,0.000212,0.583858,-1.369897


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,True,0.661458,0.676922,0.000232,0.677103,-0.29541


#### 5. Naive Bayes Classifier

In [26]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.365088,0.554229,0.000092,0.623348,-0.900447
False,0.319139,0.549435,0.000176,0.670328,-0.666240


In [27]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.365088,0.554229,0.000092,0.623348,-0.900447


In [28]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)

In [29]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.33321,0.620976,0.000409,0.645867,-1.347839


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.288962,0.529227,0.000362,0.543832,-1.528051


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.454817,0.583686,0.000111,0.597517,-0.453544


#### 6. Decision Tree

In [30]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  roc_auc_score_var  \
max_depth oversampling                                               
3         True          0.392804       0.542501           0.000139   
          False         0.075554       0.510532           0.000017   
5         True          0.341760       0.545742           0.000185   
          False         0.126115       0.519023           0.000076   
7         True          0.322796       0.539620           0.000201   
          False         0.192793       0.528527           0.000057   

                        accuracy_score  explained_variance_score  
max_depth oversampling                                            
3         True                0.523443                 -1.032145  
          False               0.728055                 -0.089144  
5         True                0.624108                 -0.855480  
          False               0.725012                 -0.153212  
7         True                0.638564                 -0.822418  
          False               0.714028                 -0.289164

In [31]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
5,True,0.34176,0.545742,0.000185,0.624108,-0.85548


In [32]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=DECISION_TREE_PARAMS)

In [33]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
7,True,0.343606,0.626825,0.000797,0.700648,-1.133938


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
7,True,0.286581,0.545329,0.000277,0.635696,-1.262888


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
5,True,0.540939,0.593472,0.000428,0.599034,-0.57223


#### 7. Random Forest

In [34]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  \
max_depth n_estimators oversampling                            
3         3            True          0.388469       0.550255   
                       False         0.037921       0.506038   
          5            True          0.374802       0.551567   
                       False         0.035918       0.506155   
          7            True          0.385000       0.553517   
                       False         0.026360       0.505205   
5         3            True          0.362149       0.547896   
                       False         0.075673       0.510541   
          5            True          0.366126       0.558653   
                       False         0.062178       0.509344   
          7            True          0.366730       0.557335   
                       False         0.054665       0.509102   
7         3            True          0.356898       0.550649   
                       False         0.136671       0.518397   
          5            True          0.354255       0.556378   
                       False         0.102243       0.514683   
          7            True          0.351786       0.558903   
                       False         0.094718       0.516878   

                                     roc_auc_score_var  accuracy_score  \
max_depth n_estimators oversampling                                      
3         3            True                   0.000079        0.564051   
                       False                  0.000008        0.731574   
          5            True                   0.000163        0.596624   
                       False                  0.000006        0.732335   
          7            True                   0.000145        0.584831   
                       False                  0.000010        0.733381   
5         3            True                   0.000165        0.608417   
                       False                  0.000019        0.728008   
          5            True                   0.000165        0.633666   
                       False                  0.000007        0.730147   
          7            True                   0.000075        0.629862   
                       False                  0.000011        0.731859   
7         3            True                   0.000117        0.626248   
                       False                  0.000048        0.720685   
          5            True                   0.000092        0.645602   
                       False                  0.000030        0.726391   
          7            True                   0.000069        0.654969   
                       False                  0.000036        0.732287   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -1.040302  
                       False                        -0.031761  
          5            True                         -0.982575  
                       False                        -0.024463  
          7            True                         -1.015150  
                       False                        -0.009088  
5         3            True                         -0.957484  
                       False                        -0.088567  
          5            True                         -0.853233  
                       False                        -0.061838  
          7            True                         -0.872143  
                       False                        -0.042542  
7         3            True                         -0.889849  
                       False                        -0.194094  
          5            True                         -0.804030  
                       False                        -0.122193  
          7            True                         -0.754033  
                       False                        -0.0702

In [35]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,7,True,0.351786,0.558903,0.000069,0.654969,-0.754033


In [36]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)

In [37]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.380664,0.650209,0.000557,0.750567,-0.869242


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,7,True,0.303532,0.554096,0.000415,0.611024,-1.338637


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.56628,0.604347,0.000263,0.608276,-0.566784


#### 8. Gradient Boosting

In [38]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  \
max_depth n_estimators oversampling                            
3         3            True          0.342981       0.545502   
                       False         0.000000       0.500000   
          5            True          0.338688       0.548528   
                       False         0.000704       0.500145   
          7            True          0.347042       0.552811   
                       False         0.004611       0.500965   
5         3            True          0.312408       0.546253   
                       False         0.001422       0.500292   
          5            True          0.297009       0.545569   
                       False         0.009795       0.502114   
          7            True          0.299847       0.548969   
                       False         0.017781       0.503264   
7         3            True          0.302600       0.542865   
                       False         0.001430       0.500197   
          5            True          0.288883       0.545599   
                       False         0.022158       0.503858   
          7            True          0.274937       0.544778   
                       False         0.043684       0.508630   

                                     roc_auc_score_var  accuracy_score  \
max_depth n_estimators oversampling                                      
3         3            True               1.954930e-04        0.618688   
                       False              0.000000e+00        0.732049   
          5            True               1.373946e-04        0.640561   
                       False              3.014514e-07        0.732097   
          7            True               1.322442e-04        0.644365   
                       False              8.800411e-07        0.732382   
5         3            True               8.468973e-05        0.667618   
                       False              2.987585e-07        0.732145   
          5            True               1.058901e-04        0.679981   
                       False              2.451338e-06        0.732858   
          7            True               1.144922e-04        0.686115   
                       False              5.386736e-06        0.732620   
7         3            True               9.039997e-05        0.670851   
                       False              1.609116e-07        0.732002   
          5            True               6.696094e-05        0.689253   
                       False              2.801173e-06        0.732430   
          7            True               3.946404e-05        0.698336   
                       False              3.717703e-06        0.734142   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -0.861881  
                       False                         0.000000  
          5            True                         -0.808935  
                       False                        -0.000153  
          7            True                         -0.804089  
                       False                        -0.000756  
5         3            True                         -0.671152  
                       False                        -0.000287  
          5            True                         -0.590495  
                       False                        -0.000973  
          7            True                         -0.556825  
                       False                        -0.008698  
7         3            True                         -0.655387  
                       False                        -0.001412  
          5            True                         -0.534054  
                       False                        -0.013571  
          7            True                         -0.464824  
                       False                        -0.0164

In [39]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,7,True,0.347042,0.552811,0.000132,0.644365,-0.804089


In [40]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)

In [41]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()[DECISION_METRIC].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,5,True,0.359555,0.639723,0.000525,0.70859,-1.092923


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,7,True,0.294287,0.557513,0.000349,0.670472,-1.094566


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.606744,0.634107,0.000375,0.636552,-0.457865


### Results

In [42]:
# display('logistic', best_log)
# display('lda', best_lda)
# display('knn', best_knn)
# display('svc', best_svc)
# display('naive_bayes', best_nb)
# display('decision_tree', best_dt)
# display('random_forest', best_rf)
# display('gradient_boosting', best_gb)

In [43]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [44]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
# display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
# display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1, best_knn_2, best_knn_3]))
# display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
# display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
# display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
# display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
# display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'knn'

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.438047       0.730204           0.000176   
            True          0.421320       0.663248           0.000327   
            True          0.689057       0.702560           0.000240   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.699028                 -0.923426  
            True                0.673097                 -0.917793  
            True                0.702483                 -0.192144

In [45]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

,models,auc_best,auc_expert_weighted
0,logistic,0.576921,0.614869
1,lda,0.577677,0.619382
2,knn,0.645125,0.696430
3,svc,0.597194,0.635002
4,naive_bayes,0.554229,0.574899
5,decision_tree,0.545742,0.585818
6,random_forest,0.558903,0.599597
7,gradient_boosting,0.552811,0.608019


### Conclusions

In [46]:
# Save dataframe
results.to_pickle("results_fine_no_metadata.pkl")

In [47]:
results.mean(axis=0)

auc_best               0.576075
auc_expert_weighted    0.616752
dtype: float64

In [48]:
results.loc[results['auc_best'].argmax()]

models                      knn
auc_best               0.645125
auc_expert_weighted     0.69643
Name: 2, dtype: object

In [49]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      knn
auc_best               0.645125
auc_expert_weighted     0.69643
Name: 2, dtype: object